In [56]:
# imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [57]:
# google trends
def merge_and_normalize_trends(dfs, col_name):
    # Start with the first DataFrame
    result = dfs[0].copy()
    for next_df in dfs[1:]:
        # Find overlap
        overlap = set(result["Woche"]) & set(next_df["Woche"])
        if overlap:
            # Use the mean ratio in the overlap to normalize
            overlap = list(overlap)
            result_overlap = result[result["Woche"].isin(overlap)].set_index("Woche")
            next_overlap = next_df[next_df["Woche"].isin(overlap)].set_index("Woche")
            ratio = result_overlap[col_name].mean() / next_overlap[col_name].mean()
        else:
            ratio = 1.0
        # Normalize next_df
        next_df_norm = next_df.copy()
        next_df_norm[col_name] = next_df_norm[col_name] * ratio
        # Append only non-overlapping part
        non_overlap = ~next_df_norm["Woche"].isin(result["Woche"])
        result = pd.concat([result, next_df_norm[non_overlap]], ignore_index=True)
    # Sort by date
    result = result.sort_values("Woche").reset_index(drop=True)
    return result

# Keyword: "Auto Finanzierung"
gt_auto_finanzierung_1 = pd.read_csv('./original_data/gt_auto_finanzierung_1.csv')
gt_auto_finanzierung_2 = pd.read_csv('./original_data/gt_auto_finanzierung_2.csv')
gt_auto_finanzierung_3 = pd.read_csv('./original_data/gt_auto_finanzierung_3.csv')

gt_auto_finanzierung = merge_and_normalize_trends(
  [gt_auto_finanzierung_1, gt_auto_finanzierung_2, gt_auto_finanzierung_3],
  col_name="Auto Finanzierung"
)
gt_auto_finanzierung.rename(columns={"Woche": "DATE"}, inplace=True)
# Keyword: "BMW Finanzierung"
gt_bmw_finanzierung_1 = pd.read_csv('./original_data/gt_bmw_finanzierung_1.csv')
gt_bmw_finanzierung_2 = pd.read_csv('./original_data/gt_bmw_finanzierung_2.csv')
gt_bmw_finanzierung_3 = pd.read_csv('./original_data/gt_bmw_finanzierung_3.csv')

gt_bmw_finanzierung = merge_and_normalize_trends(
  [gt_bmw_finanzierung_1, gt_bmw_finanzierung_2, gt_bmw_finanzierung_3],
  col_name="BMW Finanzierung"
)
gt_bmw_finanzierung.rename(columns={"Woche": "DATE"}, inplace=True)
# Keyword: "Auto Leasing"
gt_auto_leasing_1 = pd.read_csv('./original_data/gt_auto_leasing_1.csv')
gt_auto_leasing_2 = pd.read_csv('./original_data/gt_auto_leasing_2.csv')
gt_auto_leasing_3 = pd.read_csv('./original_data/gt_auto_leasing_3.csv')

gt_auto_leasing = merge_and_normalize_trends(
  [gt_auto_leasing_1, gt_auto_leasing_2, gt_auto_leasing_3],
  col_name="Auto Leasing"
)
gt_auto_leasing.rename(columns={"Woche": "DATE"}, inplace=True)
# Keyword: "BMW Leasing"
gt_bmw_leasing_1 = pd.read_csv('./original_data/gt_bmw_leasing_1.csv')
gt_bmw_leasing_2 = pd.read_csv('./original_data/gt_bmw_leasing_2.csv')
gt_bmw_leasing_3 = pd.read_csv('./original_data/gt_bmw_leasing_3.csv')
gt_bmw_leasing = merge_and_normalize_trends(
  [gt_bmw_leasing_1, gt_bmw_leasing_2, gt_bmw_leasing_3],
  col_name="BMW Leasing"
)
gt_bmw_leasing.rename(columns={"Woche": "DATE"}, inplace=True)
# Keyword: "Leasing"
gt_leasing_1 = pd.read_csv('./original_data/gt_leasing_1.csv')
gt_leasing_2 = pd.read_csv('./original_data/gt_leasing_2.csv')
gt_leasing_3 = pd.read_csv('./original_data/gt_leasing_3.csv')
gt_leasing = merge_and_normalize_trends(
  [gt_leasing_1, gt_leasing_2, gt_leasing_3],
  col_name="leasing"
)
gt_leasing.rename(columns={"Woche": "DATE"}, inplace=True)
# Keyword: "BMW"
gt_bmw_1 = pd.read_csv('./original_data/gt_bmw_1.csv')
gt_bmw_2 = pd.read_csv('./original_data/gt_bmw_2.csv')
gt_bmw_3 = pd.read_csv('./original_data/gt_bmw_3.csv')
gt_bmw = merge_and_normalize_trends(
  [gt_bmw_1, gt_bmw_2, gt_bmw_3],
  col_name="BMW"
)
gt_bmw.rename(columns={"Woche": "DATE"}, inplace=True)


In [58]:
# apgi
apgi = pd.read_csv('./original_data/apgi.csv')

apgi['DATE'] = pd.to_datetime(apgi['DATE'], format='%Y-%m') + pd.offsets.MonthEnd(0)
apgi['DATE'] = apgi['DATE'].dt.strftime('%Y-%m-%d')

In [59]:
# arbeitslosenzahlen
arbeitslosenquote = pd.read_csv("./original_data/arbeitslosenzahlen.csv", sep=";", decimal=",")

# Clean integer columns (remove dots, convert to int)
for col in ["Bestand Arbeitslose", "Zugang Arbeitslose", "Abgang Arbeitslose"]:
  arbeitslosenquote[col] = arbeitslosenquote[col].astype(str).str.replace(".", "", regex=False).replace("", "0").astype(int)

# Clean float column (replace comma with dot, convert to float)
arbeitslosenquote["Arbeitslosenquote"] = arbeitslosenquote["Arbeitslosenquote"].astype(str).str.replace(",", ".", regex=False).replace("", "0").astype(float)

# Convert 'Berichtsmonat' to datetime
arbeitslosenquote["DATE"] = arbeitslosenquote["Berichtsmonat"].astype(str).apply(
  lambda x: pd.to_datetime(
    f"{x.split(' ')[1]}-{month_map.get(x.split(' ')[0], '01')}-01", errors="coerce"
  )
)
arbeitslosenquote["DATE"] = arbeitslosenquote["DATE"] + pd.offsets.MonthEnd(0)

# Reorder columns
arbeitslosenquote = arbeitslosenquote.drop(columns=["Berichtsmonat"])
arbeitslosenquote = arbeitslosenquote[["DATE"] + [col for col in arbeitslosenquote.columns if col != "DATE"]]

arbeitslosenquote


,DATE,Bestand Arbeitslose,Zugang Arbeitslose,Abgang Arbeitslose,Arbeitslosenquote
0,2025-04-30,2931550,562010,597530,6.3
1,2025-03-31,2967080,577380,599530,6.4
2,2025-02-28,2989220,585600,589050,6.4
3,2025-01-31,2992660,625980,439960,6.4
4,2024-12-31,2806630,562260,529670,6.0
...,...,...,...,...,...
211,2007-09-30,3530640,673130,834200,8.4
212,2007-08-31,3691890,671520,680410,8.8
213,2007-07-31,3701000,775270,746800,8.8
214,2007-06-30,3672930,578000,700850,8.8


In [60]:
# euribor
# 3 months
euribor_3m = pd.read_csv('./original_data/euribor_3m.csv')
euribor_3m = euribor_3m.drop(columns=['TIME PERIOD'])
euribor_3m = euribor_3m.rename(
    columns={
        "Euribor 3-month - Historical close, average of observations through period (FM.M.U2.EUR.RT.MM.EURIBOR3MD_.HSTA)": "Euribor 3M"
    }
)
# 6 months
euribor_6m = pd.read_csv('./original_data/euribor_6m.csv')
euribor_6m = euribor_6m.drop(columns=['TIME PERIOD'])
euribor_6m = euribor_6m.rename(
    columns={
        "Euribor 6-month - Historical close, average of observations through period (FM.M.U2.EUR.RT.MM.EURIBOR6MD_.HSTA)": "Euribor 6M"
    }
)
# 12 months
euribor_12m = pd.read_csv('./original_data/euribor_12m.csv')
euribor_12m = euribor_12m.drop(columns=['TIME PERIOD'])
euribor_12m = euribor_12m.rename(
    columns={
        "Euribor 1-year - Historical close, average of observations through period (FM.M.U2.EUR.RT.MM.EURIBOR1YD_.HSTA)": "Euribor 12M"
    }
)


In [61]:
# gebrauchtwagenpreiseindex

gebrauchtwagenpreisindex = pd.read_csv(
    "./original_data/gebrauchtwagenpreisindex.csv"
)
gebrauchtwagenpreisindex["DATE"] = pd.to_datetime(
    gebrauchtwagenpreisindex["DATE"], errors="coerce"
) + pd.offsets.MonthEnd(0)
gebrauchtwagenpreisindex["DATE"] = gebrauchtwagenpreisindex["DATE"].dt.strftime(
    "%Y-%m-%d"
)

gebrauchtwagenpreisindex

,DATE,Gebrauchtwagenpreisindex
0,2015-01-31,100.0
1,2015-02-28,108.4
2,2015-03-31,111.4
3,2015-04-30,112.2
4,2015-05-31,109.7
...,...,...
119,2024-12-31,137.3
120,2025-01-31,135.2
121,2025-02-28,137.0
122,2025-03-31,138.8


In [62]:
# ifo beschäftigungsbarometer
beschaeftigungsbarometer = pd.read_csv(
    "./original_data/ifo_beschaeftigungsbarometer.csv"
)
beschaeftigungsbarometer["DATE"] = pd.to_datetime(
    beschaeftigungsbarometer["DATE"], errors="coerce"
) + pd.offsets.MonthEnd(0)
beschaeftigungsbarometer["DATE"] = beschaeftigungsbarometer["DATE"].dt.strftime(
    "%Y-%m-%d"
)

beschaeftigungsbarometer 

,DATE,Beschäftigungsbarometer,Verarbeitendes Gewerbe,Bauhauptgewerbe,Handel,Dienstleistungssektor
0,2002-01-31,91.9,-23.7,-45.4,-21.6,2.0
1,2002-02-28,92.1,-21.5,-37.2,-22.9,1.5
2,2002-03-31,92.5,-22.1,-34.4,-19.9,1.9
3,2002-04-30,92.0,-19.4,-38.3,-22.9,0.1
4,2002-05-31,92.1,-20.2,-36.8,-22.6,0.8
...,...,...,...,...,...,...
275,2024-12-31,92.5,-22.4,-2.0,-16.7,-2.4
276,2025-01-31,93.4,-22.3,-3.7,-15.6,1.0
277,2025-02-28,93.0,-19.6,-4.5,-15.9,-1.4
278,2025-03-31,92.8,-20.8,-1.8,-12.9,-2.8


In [63]:
# ifo geschäftsklima
geschaeftsklima = pd.read_csv("./original_data/ifo_geschaeftsklima.csv")
geschaeftsklima["DATE"] = pd.to_datetime(
  geschaeftsklima["DATE"].str.strip(), format="%m/%Y"
) + pd.offsets.MonthEnd(0)
geschaeftsklima["DATE"] = geschaeftsklima["DATE"].dt.strftime("%Y-%m-%d")
geschaeftsklima


,DATE,Geschäftsklima,Geschäftslage,Geschäftserwartungen,Konjunkturampel
0,2005-01-31,92.2,87.4,97.2,83.1
1,2005-02-28,92.0,88.0,96.2,50.4
2,2005-03-31,90.1,85.9,94.5,4.9
3,2005-04-30,89.9,86.3,93.7,18.7
4,2005-05-31,89.3,86.1,92.7,11.7
...,...,...,...,...,...
239,2024-12-31,84.7,85.0,84.4,13.0
240,2025-01-31,85.2,86.0,84.3,26.9
241,2025-02-28,85.3,85.0,85.6,30.2
242,2025-03-31,86.7,85.7,87.7,69.9


In [64]:
# bmw sales
bmw_sales = pd.read_csv("./original_data/bmw_sales.csv", sep=";", decimal=",")

In [65]:
# Merge all dataframes on DATE
from functools import reduce
dataframes = [
    bmw_sales,
    gt_auto_finanzierung,
    gt_bmw_finanzierung,
    gt_auto_leasing,
    gt_bmw_leasing,
    gt_leasing,
    gt_bmw,
    apgi,
    gebrauchtwagenpreisindex,
    euribor_3m,
    euribor_6m,
    euribor_12m,
    arbeitslosenquote,
    beschaeftigungsbarometer,
    geschaeftsklima,
]

# Ensure all DATE columns are in datetime format
for df_item in dataframes:
    if "DATE" in df_item.columns:
        df_item["DATE"] = pd.to_datetime(df_item["DATE"])

merged = reduce(lambda left, right: pd.merge(left, right, on="DATE", how="outer"), dataframes)
merged.to_csv("data.csv", index=False)
merged


,DATE,SALES,Auto Finanzierung,BMW Finanzierung,Auto Leasing,BMW Leasing,leasing,BMW,AGPI,Gebrauchtwagenpreisindex,...,Arbeitslosenquote,Beschäftigungsbarometer,Verarbeitendes Gewerbe,Bauhauptgewerbe,Handel,Dienstleistungssektor,Geschäftsklima,Geschäftslage,Geschäftserwartungen,Konjunkturampel
0,1994-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1994-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1994-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1994-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1994-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897,2025-04-27,3046.0,100.252569,63.948932,139.063281,82.585880,111.847406,80.010830,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
898,2025-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.9,...,6.3,93.9,-16.9,-4.0,-11.8,-0.3,86.9,86.4,87.4,70.6
899,2025-05-04,3784.0,95.589659,52.002428,142.374311,84.848507,111.847406,80.010830,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,2025-05-11,3775.0,111.909844,54.110634,148.996372,90.505074,128.541049,86.245440,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Data transformation
# Remove all whitespace from column names
merged.columns = merged.columns.str.replace(r"\s+", "", regex=True)

# Adding Weeks
# Add missing weekly dates if not present
all_weeks = pd.date_range(start=merged['DATE'].min(), end=merged['DATE'].max(), freq='W-SUN')
missing_weeks = all_weeks.difference(merged['DATE'])
if not missing_weeks.empty:
  missing_df = pd.DataFrame({'DATE': missing_weeks})
  merged = pd.concat([merged, missing_df], ignore_index=True)
  merged = merged.sort_values('DATE').reset_index(drop=True)

  # Keep only rows where DATE is in all_weeks (i.e., weekly dates)
merged = merged[merged['DATE'].isin(all_weeks)].reset_index(drop=True)

# Remove all data before 2015
merged = merged[merged['DATE'] >= pd.Timestamp('2014-01-01')].reset_index(drop=True)




,DATE,SALES,AutoFinanzierung,BMWFinanzierung,AutoLeasing,BMWLeasing,leasing,BMW,AGPI,Gebrauchtwagenpreisindex,...,Arbeitslosenquote,Beschäftigungsbarometer,VerarbeitendesGewerbe,Bauhauptgewerbe,Handel,Dienstleistungssektor,Geschäftsklima,Geschäftslage,Geschäftserwartungen,Konjunkturampel
0,2015-01-04,5555.0,68.000000,55.000000,48.000000,69.000000,52.000000,69.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-11,5040.0,67.000000,67.000000,56.000000,73.000000,56.000000,79.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-18,3365.0,89.000000,88.000000,62.000000,76.000000,63.000000,81.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-25,4050.0,70.000000,70.000000,58.000000,75.000000,59.000000,76.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-02-01,3984.0,82.000000,79.000000,62.000000,72.000000,57.000000,83.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,2025-04-20,3042.0,106.081206,64.651667,145.685342,91.636387,120.194227,87.284542,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538,2025-04-27,3046.0,100.252569,63.948932,139.063281,82.585880,111.847406,80.010830,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
539,2025-05-04,3784.0,95.589659,52.002428,142.374311,84.848507,111.847406,80.010830,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
540,2025-05-11,3775.0,111.909844,54.110634,148.996372,90.505074,128.541049,86.245440,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
# Interpolating missing values
interpolated = merged.interpolate(method='linear', limit_direction='both')

interpolated = interpolated[
    interpolated["DATE"] >= pd.Timestamp("2014-01-01")
].reset_index(drop=True)

interpolated

,DATE,SALES,AutoFinanzierung,BMWFinanzierung,AutoLeasing,BMWLeasing,leasing,BMW,AGPI,Gebrauchtwagenpreisindex,...,Arbeitslosenquote,Beschäftigungsbarometer,VerarbeitendesGewerbe,Bauhauptgewerbe,Handel,Dienstleistungssektor,Geschäftsklima,Geschäftslage,Geschäftserwartungen,Konjunkturampel
0,2015-01-04,5555.0,68.000000,55.000000,48.000000,69.000000,52.000000,69.000000,20648.0,109.7,...,6.3,100.2,2.3,-5.0,-0.6,11.8,100.4,100.5,100.2,96.1
1,2015-01-11,5040.0,67.000000,67.000000,56.000000,73.000000,56.000000,79.000000,20648.0,109.7,...,6.3,100.2,2.3,-5.0,-0.6,11.8,100.4,100.5,100.2,96.1
2,2015-01-18,3365.0,89.000000,88.000000,62.000000,76.000000,63.000000,81.000000,20648.0,109.7,...,6.3,100.2,2.3,-5.0,-0.6,11.8,100.4,100.5,100.2,96.1
3,2015-01-25,4050.0,70.000000,70.000000,58.000000,75.000000,59.000000,76.000000,20648.0,109.7,...,6.3,100.2,2.3,-5.0,-0.6,11.8,100.4,100.5,100.2,96.1
4,2015-02-01,3984.0,82.000000,79.000000,62.000000,72.000000,57.000000,83.000000,20648.0,109.7,...,6.3,100.2,2.3,-5.0,-0.6,11.8,100.4,100.5,100.2,96.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,2025-04-20,3042.0,106.081206,64.651667,145.685342,91.636387,120.194227,87.284542,26656.0,138.2,...,5.8,95.8,-10.2,-5.1,-12.7,4.2,88.4,88.2,88.5,64.7
538,2025-04-27,3046.0,100.252569,63.948932,139.063281,82.585880,111.847406,80.010830,26656.0,138.2,...,5.8,95.8,-10.2,-5.1,-12.7,4.2,88.4,88.2,88.5,64.7
539,2025-05-04,3784.0,95.589659,52.002428,142.374311,84.848507,111.847406,80.010830,26656.0,138.2,...,5.8,95.8,-10.2,-5.1,-12.7,4.2,88.4,88.2,88.5,64.7
540,2025-05-11,3775.0,111.909844,54.110634,148.996372,90.505074,128.541049,86.245440,26656.0,138.2,...,5.8,95.8,-10.2,-5.1,-12.7,4.2,88.4,88.2,88.5,64.7


In [ ]:
# seasonal adjustment
from statsmodels.tsa.seasonal import STL

# List of columns to be seasonally adjusted
seasonal_cols = [
    "SALES",
    "Auto Leasing",
    "Auto Finanzierung",
    "BMW Leasing",
    "BMW Finanzierung",
    "BMW",
    "Gebrauchtwagenpreisindex",
    "Bestand Arbeitslose",
    "Zugang Arbeitslose",
    "Abgang Arbeitslose",
    "Arbeitslosenquote",
    "Verbraucherpreisindex",
    "Index der Einzelhandelspreise",
    "Index der Erzeugerpreise gewerblicher Produkte",
    "Index der Growsshandelsverkaufspreise",
    "Index der Einfuhrpreise",
    "Index der Ausfuhrpreise",
]

df_sa = interpolated.copy()
df_sa = df_sa.set_index("DATE")

for col in seasonal_cols:
    if col in df_sa.columns:
        series = df_sa[col].dropna()  # Remove NaNs
        if len(series) > 2 * 12:  # Check if enough data points (period=12)
            stl = STL(series, period=12, robust=True)
            res = stl.fit()
            # Place the seasonally adjusted data back, aligning with the original index
            df_sa[col] = pd.Series(res.trend + res.resid, index=series.index)
        else:
            print(
                f"Column '{col}' has insufficient data points ({len(series)}) for STL decomposition after dropping NaNs. Skipping."
            )
            # Optionally, fill with NaNs or keep original if not enough data
            df_sa[col] = np.nan
    else:
        print(f"Warning: Column '{col}' not found in DataFrame df_sa.")

df_sa = df_sa.reset_index()

df_sa